### *PhotoDissociation Region Toolbox Notebooks*
-------------------------------------------------------------

# Example 3: Determining Radiation Field and Density

This example shows use the PDRT Toolbox to determine the PDR radiation field $G_0$ and hydrogen nucleus volume density $n$ from your spectral line and far-infrared (FIR) data into the PDR Toolbox.  The case is for single-pixel observations (as in the classic PDRT website).   If you have not gone through the Measurements and ModelSets examples, you should do them first.

[`LineRatioFit`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.tool.html#lineratiofit) is a tool to fit observations of intensity ratios to a set of PDR models. It takes as input a set of observations with errors represented as [`Measurements`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.measurement.html) and  [`ModelSet`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.modelset.html) for the models to which the data will be fitted. The observations should be spectral line or continuum intensities.  They can be spatial maps or single pixel values. They should have the same spatial resolution.  

The models to be fit are stored as intensity ratios.  The input observations will be use to create ratios that correspond to models.  From there a minimization fit is done to determine the density and radiation field that best fit the data.At least 3 observations are needed in order to make at least 2 ratios.  With fewer ratios, no fitting can be done. More ratios generally means better determined density and radiation field, assuming the data are consistent with each other.   

Once the fit is done, [`LineRatioPlot`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.plot.html#lineratioplot) can be used to view the results.


### Radiation Field and Density from single value Measurements
Following the example on how to use Measurements, create Measurements for your observations.

In [ ]:
from pdrtpy.measurement import Measurement
from pdrtpy.modelset import ModelSet
import pdrtpy.pdrutils as utils
from pdrtpy.tool.lineratiofit import LineRatioFit
from pdrtpy.plot.lineratioplot import LineRatioPlot
from astropy.nddata import StdDevUncertainty
import astropy.units as u
import numpy as np

In [ ]:
# check that your notebooks align with your pdrtpy version
utils.check_nb()

In [ ]:
myunit = "erg s-1 cm-2 sr-1" # my default unit for value and error
m1 = Measurement(data=6E-6,uncertainty = StdDevUncertainty(1E-6),identifier="OI_145",unit=myunit)
m2 = Measurement(data=2.2E-6,uncertainty = StdDevUncertainty(5E-7),identifier="CI_609",unit=myunit)
m3 = Measurement(data=260.0,uncertainty = StdDevUncertainty(40.0),identifier="CO_21",restfreq="230.538 GHz", unit="K km/s")
m4 = Measurement(data=2E-5,uncertainty = StdDevUncertainty(2E-6),identifier="CII_158",unit=myunit)
a = [m1,m2,m3,m4]

Now create the fitting tool, feeding it your observations.   It will use the default  Wolfire/Kauffman ModelSet.

In [ ]:
p = LineRatioFit(ModelSet("wk2006",z=1),measurements=a)

### Now run it! 
Note the K km s$^{-1}$ get converted on the fly to erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$.  You will get warned that there are no beam parameters in the Measurements.

In [ ]:
p.run()

### The results are stored in member variables as Measurements.
`pdrutils` has methods to convert between the common radiation field measures. Note: the errors in $n$ and $G_0$ have been set to `NaN` because we don't have a good way to calculate them yet.

In [ ]:
print(f"Density = {p.density:3.2e}")
print(f"Radiation Field = {p.radiation_field:3.2e}")
# example conversions
print(f"{utils.toDraine(p.radiation_field):3.2f}")
print(f"{utils.tocgs(p.radiation_field):3.2e}")
print(f"{utils.toMathis(p.radiation_field):3.2e}")

## Now on to plotting!
Create a plotter from the tool.  For single pixel measurements, you can plot the observed ratios in (G0,n) space like the classic PDRT.  

In [ ]:
plot = LineRatioPlot(p)

In [ ]:
# since you only have one pixel, asking to plot density, radiation field, or both just prints
plot.show_both()

### LineRatioPlot has many options for how plots are displayed.
You can vary colormap, contours, units, etc.  For an exhaustive list, see the documentation web page or type `help(LineRatioPlot)`.
    

#### Plot your observed ratios with errors on the matching models.  
Here we also show how to change the figure size.

In [ ]:
plot.ratios_on_models(yaxis_unit="Habing",image=True,norm='simple',ncols=1,figsize=(10,20), meas_color=['red'])
# Save the figure to a PNG
plot.savefig("modelfits.png")


In [ ]:
plot.ratios_on_models(norm='log',label=True,cmap='rainbow',shading=0)
# Save as a PNG file
plot.savefig("compare.png")

#### Plot the reduced $\chi^2$ in $(n,G_0)$ space, using an alternative colormap and label the contours

In [ ]:
plot.reduced_chisq(cmap='gray_r',norm='log',label=True,colors='white',legend=True,vmax=8E4,figsize=(4,4),yaxis_unit='Habing')
# save as a PNG file
plot.savefig("chisq.png")

#### How about just contours? We need to specify color since default contour color is white.  Also add a legend showing the values at the minimum.

In [ ]:
plot.chisq(image=False,cmap='gray_r',norm='log',colors='k',label=True,legend=True,yaxis_unit='Habing')

#### Plot confidence intervals.
The default levels are [50, 68, 80, 95, 99]

In [ ]:
plot.confidence_intervals()

#### What are the model ratios matching these observations?

In [ ]:
p._modelratios.keys()

#### Plot one of the model ratios

In [ ]:
plot.modelratio("CII_158/CI_609")

#### Save the most recent figure to a PNG

In [ ]:
plot.savefig("CII_CI.png")

#### Overlay all the ratios and errors in model space.

In [ ]:
plot.overlay_all_ratios(yaxis_unit="Habing",figsize=(5,5))

# What if want to fit multiple sets of measurements that aren't in map form?
You can do a bulk import and fit by
reading in tables that contain intensities for individual lines to 
create a `Measurement` instance containing a 1-D array of data points using [*Measurement.from_table()*.](http://pdrtpy.readthedocs.io/en/latest/pdrtpy.measurement.html#pdrtpy.measurement.Measurement.from_table)  

Below is an example using \[C II\], CO(3-2), and FIR data on the source RCW 49 from [Tiwari et al. 2021.](https://ui.adsabs.harvard.edu/abs/2021ApJ...914..117T/abstract)  In this example, the tables are in IPAC format, but any Astropy supported table format is acceptable.  

In [ ]:
m1 = Measurement.from_table("rcw49_nc_cii158.tab")
m2 = Measurement.from_table("rcw49_nc_co32.tab")
m3 = Measurement.from_table("rcw49_nc_fir.tab")
ms = ModelSet("wk2006",z=1)
lrf = LineRatioFit(ms,measurements=[m1,m2,m3])
# run it
lrf.run()

Plotting in `LineRatioPlot` is set up to handle single pixels or spatial maps, rather than vectors. But you can examine the inputs and results in a table.  The `table` property returns the inputs and fits in an astropy Table.   If you have not yet called `run()` it will include only your input `Measurements`.

In [ ]:
t=lrf.table
t.show_in_notebook()

#### From here you can use matplotlib tools to explore the data further.
For example, below is a plot of two intensity ratios.

In [ ]:
import matplotlib.pyplot as plt
x='CII_158/FIR'
y='CII_158/CO_32'
plt.scatter(t[x],t[y])
plt.xlabel(x)
plt.ylabel(y)

#### This shows a scatter plot of CII 158 $\mu$m intensity vs. $G_0$.   
The data are quantized to the model grid.

In [ ]:
x='CII_158'
y='Radiation Field'
plt.scatter(t[x],t[y])
plt.xlabel(f'{x} [{t[x].unit}]')
plt.ylabel(f'{y} [{t[y].unit}]')

#### And here is $\chi^2$ vs. $G_0$

In [ ]:
y='Radiation Field'
x='Chi-square'
plt.scatter(t[x],t[y])
plt.xlabel(r'$\chi^2$')
plt.ylabel(r'$G_0$'+f' [{t[y].unit}]')